In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import regex as re
import math
from collections import defaultdict
import pytz
import shin

from scipy.stats import norm
from scipy.optimize import brentq
from scipy.special import expit, logit

from rapidfuzz.fuzz import ratio

things to look at: how accurate is the prematch probability when it ends (if a team has over 50% chance to win, how often do they actually win the game?)

if my fair probability is wildely different than the midprice, should I weigh it less or find a new fair probability? How often is my direction right?

can i still find edge in a very wide market?

In [5]:
date = '2025-12-01'
odds_sport = 'nfl' #cbb, cfb, nba, nfl
kalshi_sport = 'nfl' #ncaab, ncaaf, nba, nfl

In [ ]:
odds_df = pd.read_csv(f"../data_collection/updated_scripts/oddsapi_outputs/{date}/{odds_sport}_odds.csv")
odds_df.drop(columns=['league'], inplace=True)
odds_df.rename(columns={'price': 'odds'}, inplace=True)
odds_df['vig_prob'] = 1 / odds_df['odds']

def remove_vig_probs_add(df):
    df = df.copy()
    df['fair_prb'] = np.nan

    grouped = df.groupby(['game_id', 'bookmaker', 'market'])

    for _, group in grouped:
        if len(group) < 2:
            continue
        probs = group['vig_prob']
        total = probs.sum()
        if total == 0:
            continue
        fair_probs = (probs / total).round(4)
        df.loc[group.index, 'fair_prb'] = fair_probs

    return df

def remove_vig_probs_shin(df):
    df = df.copy()
    df['fair_prb'] = np.nan

    grouped = df.groupby(['game_id', 'bookmaker', 'market'])

    for _, group in grouped:
        if len(group) < 2:
            continue
        assert len(group) == 2
        odds = group['odds'].values
        fair_probs = shin.calculate_implied_probabilities(odds)
        df.loc[group.index, 'fair_prb'] = fair_probs

    return df

def devig_probit(p1, p2):
    if p1 <= 0 or p2 <= 0 or p1 >= 1 or p2 >= 1:
        total = p1 + p2
        return p1 / total, p2 / total
    if p1 + p2 <= 1:
        total = p1 + p2
        return p1 / total, p2 / total
    z1 = norm.ppf(p1)
    z2 = norm.ppf(p2)
    f = lambda lam: norm.cdf(z1 - lam) + norm.cdf(z2 - lam) - 1
    try:
        lam = brentq(f, -15, 15)
    except ValueError:
        total = p1 + p2
        return p1 / total, p2 / total
    q1 = norm.cdf(z1 - lam)
    q2 = norm.cdf(z2 - lam)
    return q1, q2

def remove_vig_probs_probit(df):
    df = df.copy()
    df['fair_prb'] = np.nan
    grouped = df.groupby(['game_id', 'bookmaker', 'market'])
    for _, group in grouped:
        if len(group) != 2:
            continue
        p1, p2 = group['vig_prob'].values
        q1, q2 = devig_probit(p1, p2)
        df.loc[group.index, 'fair_prb'] = [q1, q2]
    return df

def devig_logit(p1, p2):
    z1 = logit(p1)
    z2 = logit(p2)
    # Solve for λ such that logistic(z1 - λ) + logistic(z2 - λ) = 1
    f = lambda lam: expit(z1 - lam) + expit(z2 - lam) - 1
    lam = brentq(f, -20, 20)
    q1 = expit(z1 - lam)
    q2 = expit(z2 - lam)
    return q1, q2

def remove_vig_probs_logit(df):
    df = df.copy()
    df['fair_prb'] = np.nan
    grouped = df.groupby(['game_id', 'bookmaker', 'market'])
    for _, group in grouped:
        if len(group) != 2:
            continue
        p1, p2 = group['vig_prob'].values
        q1, q2 = devig_logit(p1, p2)
        df.loc[group.index, 'fair_prb'] = [q1, q2]
    return df

odds_df = remove_vig_probs_probit(odds_df)

odds_winners_df = odds_df[odds_df['market'] == 'h2h'].copy()
odds_spreads_df = odds_df[odds_df['market'] == 'spreads'].copy()
odds_spreads_df = odds_spreads_df.loc[(odds_spreads_df['point'].notna()) & (odds_spreads_df['point'] > 0)]
odds_totals_df  = odds_df[odds_df['market'] == 'totals'].copy()

# Average per-team fair probabilities across DraftKings/FanDuel/Pinnacle for winners_df
WEIGHTS = {
    "Pinnacle": 0.3,
    "BetOnline.ag": 0.3,
    "BetUS": 0.2,
    "FanDuel": 0.2
}

def wavg(x, df):
    idx = x.index
    bookmakers = df.loc[idx, 'bookmaker']
    w = np.array([WEIGHTS[b] for b in bookmakers])
    return np.average(x.values, weights=w)

mask = odds_winners_df['fair_prb'].notna()
avg_by_team = (
    odds_winners_df.loc[mask]
    .groupby(['game_id', 'team'])['fair_prb']
    .transform(lambda x: wavg(x, odds_winners_df))
    .round(4)
)
odds_winners_df.loc[mask, 'avg_fair_prb'] = avg_by_team
odds_winners_df.loc[~mask, 'avg_fair_prb'] = pd.NA

#Average fair probabilities for spreads for same game, point spread, and team
mask = odds_spreads_df['fair_prb'].notna()
avg_by_point = (
    odds_spreads_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform(lambda x: wavg(x, odds_spreads_df))
    .round(4)
)
odds_spreads_df['avg_fair_prb'] = avg_by_point

#Average fair probabilities for totals for same game, point spread, direction (Over/Under)
mask = odds_totals_df['fair_prb'].notna()
avg_by_tot_point = (
    odds_totals_df.loc[mask]
    .groupby(['game_id', 'point', 'team'])['fair_prb']
    .transform(lambda x: wavg(x, odds_totals_df))
    .round(4)
)
odds_totals_df['avg_fair_prb'] = avg_by_tot_point

In [7]:
kalshi_winners_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_winners.csv")
kalshi_totals_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_totals.csv")
kalshi_spreads_df = pd.read_csv(f"../data_collection/updated_scripts/kalshi_data_logs/{date}/{kalshi_sport}_spreads.csv")

if (kalshi_sport == 'ncaaf') | (kalshi_sport == 'nfl'):
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)
elif (kalshi_sport == 'ncaab') | (kalshi_sport == 'nba'):
    kalshi_spreads_df['points'] = kalshi_spreads_df['title'].str.extract(r'over ([\d.]+) Points\?').astype(float)
    kalshi_totals_df["points"] = kalshi_totals_df["ticker"].str.extract(r"-([0-9.]+)$").astype(float)

columns_to_drop = ['timestamp', 'market_type', 'yes_bid2', 'yes_ask2', 'no_bid2', 'no_ask2', 'yes_depth_bids', 'yes_depth_asks', 'no_depth_bids', 'no_depth_asks']
kalshi_winners_df.drop(columns=columns_to_drop, inplace=True)
kalshi_spreads_df.drop(columns=columns_to_drop, inplace=True)
kalshi_totals_df.drop(columns=columns_to_drop, inplace=True)

In [8]:
odds_totals_df.head()

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
4,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,FanDuel,totals,Over,1.91,46.5,New England Patriots,New York Giants,0.52356,0.5,0.498
5,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,FanDuel,totals,Under,1.91,46.5,New England Patriots,New York Giants,0.52356,0.5,0.502
10,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,BetUS,totals,Over,1.91,46.5,New England Patriots,New York Giants,0.52356,0.5,0.498
11,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,BetUS,totals,Under,1.91,46.5,New England Patriots,New York Giants,0.52356,0.5,0.502
16,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,BetOnline.ag,totals,Over,1.91,46.5,New England Patriots,New York Giants,0.52356,0.5,0.498


In [9]:
#get names from kalshi_winners_df
def extract_teams_from_winners(title):
    title = title.replace(" Winner?", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
    elif " vs " in title:
        right, left = title.split(" vs ", 1)
    else:
        return pd.Series([None, None])  
    left = re.sub(r'\bSt\.$', 'St', left.strip())
    right = re.sub(r'\bSt\.$', 'St', right.strip())
    return pd.Series([left, right])

kalshi_winners_df[['home_team', 'away_team']] = kalshi_winners_df['title'].apply(extract_teams_from_winners)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_winners_teams = flat_teams.tolist()

#get names from kalshi_totals_df
def extract_teams_from_totals(title):
    title = title.replace(": Total Points", "")
    if " at " in title:
        right, left = title.split(" at ", 1)
        left = re.sub(r'\bSt\.$', 'St', left.strip())
        right = re.sub(r'\bSt\.$', 'St', right.strip())
        return pd.Series([left, right])
    return None

kalshi_totals_df[['home_team', 'away_team']] = kalshi_totals_df['title'].apply(extract_teams_from_totals)
unique_rows = kalshi_winners_df.drop_duplicates(subset=['home_team', 'away_team'])
flat_teams = pd.unique(unique_rows[['home_team', 'away_team']].values.ravel())
kalshi_totals_teams = flat_teams.tolist()

#get names from kalshi_spreads_df
def extract_team_from_spreads(title):
    if " wins by " in title:
        team = title.split(" wins by ", 1)[0].strip()
        team = re.sub(r'\bSt\.$', 'St', team)
        return team
    return None

kalshi_spreads_df['team'] = kalshi_spreads_df['title'].apply(extract_team_from_spreads)
unique_teams_spread = kalshi_spreads_df['team'].drop_duplicates()
kalshi_spreads_teams = unique_teams_spread.tolist()

In [10]:
odds_teams_by_market = odds_df.groupby('market')['team'].unique().to_dict()

def fuzzy_match_kalshi_to_odds(kalshi_teams, odds_team_names):
    matched_kalshi = []
    matched_odds = []
    candidates_dict = defaultdict(list)

    kalshi_sorted = sorted(kalshi_teams, key=lambda x: x[0] if x else '')
    remaining_odds = sorted(odds_team_names.tolist().copy())

    for kalshi_name in kalshi_sorted:
        candidates = []
        for odds_name in remaining_odds:
            if kalshi_name in odds_name:
                candidates.append(odds_name)
        if len(candidates) == 1:
            candidates_dict[candidates[0]].append(kalshi_name)
        elif len(candidates) > 1:
            best_fit = candidates[0]
            best_ratio = ratio(best_fit, kalshi_name)
            for name in candidates:
                curr_ratio = ratio(name, kalshi_name)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
            candidates_dict[best_fit].append(kalshi_name)
    
    for odd, kalsh in candidates_dict.items():
        best_fit = kalsh[0]
        best_ratio = ratio(best_fit, odd)
        if len(kalsh) > 1:
            for name in kalsh:
                curr_ratio = ratio(name, odd)
                if curr_ratio > best_ratio:
                    best_fit = name
                    best_ratio = curr_ratio
        matched_odds.append(odd)
        matched_kalshi.append(best_fit)


    return matched_kalshi, matched_odds


# Winners / h2h
matched_kalshi_h2h, matched_odds_h2h = fuzzy_match_kalshi_to_odds(
    kalshi_winners_teams,
    odds_teams_by_market.get('h2h', [])
)

# Spreads
matched_kalshi_spreads, matched_odds_spreads = fuzzy_match_kalshi_to_odds(
    kalshi_spreads_teams,
    odds_teams_by_market.get('spreads', [])
)

# Totals (match only Over/Under)
totals_odds_df = odds_df[odds_df['market'] == 'totals']
odds_totals_teams = pd.unique(totals_odds_df[['home_team', 'away_team']].values.ravel())
matched_kalshi_totals, matched_odds_totals = fuzzy_match_kalshi_to_odds(
    kalshi_totals_teams,
    odds_totals_teams
)

matched_names = {
    'h2h': {
        'kalshi': matched_kalshi_h2h,
        'odds': matched_odds_h2h
    },
    'spreads': {
        'kalshi': matched_kalshi_spreads,
        'odds': matched_odds_spreads
    },
    'totals': {
        'kalshi': matched_kalshi_totals,
        'odds': matched_odds_totals
    }
}


In [11]:
assert(len(matched_names['h2h']['kalshi']) == len(matched_names['h2h']['odds']))
assert(len(matched_names['spreads']['kalshi']) == len(matched_names['spreads']['odds']))
assert(len(matched_names['totals']['kalshi']) == len(matched_names['totals']['odds']))

In [12]:
odds_winners_df = odds_winners_df[
    odds_winners_df['home_team'].isin(matched_names['h2h']['odds']) |
    odds_winners_df['away_team'].isin(matched_names['h2h']['odds'])
].drop_duplicates(subset='team').sort_values(by='home_team').reset_index(drop=True)

kalshi_winners_df = kalshi_winners_df[
    kalshi_winners_df['home_team'].isin(matched_names['h2h']['kalshi']) |
    kalshi_winners_df['away_team'].isin(matched_names['h2h']['kalshi'])
].sort_values(by='home_team').reset_index(drop=True)

odds_spreads_df = odds_spreads_df[odds_spreads_df['team'].isin(matched_names['spreads']['odds'])].reset_index(drop=True)
kalshi_spreads_df = kalshi_spreads_df[kalshi_spreads_df['team'].isin(matched_names['spreads']['kalshi'])].reset_index(drop=True)

odds_totals_df = odds_totals_df[
    odds_totals_df['home_team'].isin(matched_names['totals']['odds']) |
    odds_totals_df['away_team'].isin(matched_names['totals']['odds'])
].sort_values(by='home_team').reset_index(drop=True)
kalshi_totals_df = kalshi_totals_df[
    (kalshi_totals_df['home_team'].isin(matched_names['totals']['kalshi'])) | 
    (kalshi_totals_df['away_team'].isin(matched_names['totals']['kalshi']))
    ].sort_values(by='home_team').reset_index(drop=True)


In [13]:
odds_winners_df.head()

,sport,game_id,start_time,bookmaker,market,team,odds,point,home_team,away_team,vig_prob,fair_prb,avg_fair_prb
0,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,FanDuel,h2h,New England Patriots,1.27,NaN,New England Patriots,New York Giants,0.787402,0.769124,0.7639
1,NFL,3e65786eddd4c7aee9e7f2504f6fdedc,2025-12-01 19:15:00 CST,FanDuel,h2h,New York Giants,4.00,NaN,New England Patriots,New York Giants,0.250000,0.230876,0.2361


In [14]:
kalshi_winners_df.head()

,ticker,title,status,event_start_time,yes_bid,yes_ask,no_bid,no_ask,yes_spread,no_spread,liquidity_dollars,volume_24h,home_team,away_team
0,KXNFLGAME-25DEC01NYGNE-NYG,New York G at New England Winner?,active,2025-12-15T20:15:00-05:00,0.24,0.25,0.75,0.76,0.01,0.01,9618831.40,672664.0,New England,New York G
1,KXNFLGAME-25DEC01NYGNE-NE,New York G at New England Winner?,active,2025-12-15T20:15:00-05:00,0.76,0.77,0.23,0.24,0.01,0.01,13164542.38,1753902.0,New England,New York G


In [15]:
# Concatenate winners df

# Specify the columns to extract
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'no_bid', 'no_ask', 'home_team', 'away_team']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb']

# Rename overlapping columns in odds to prevent clashes
odds_subset = odds_winners_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team'
})

kalshi_subset = kalshi_winners_df[kalshi_cols].rename(columns={
    'home_team': 'kalshi_home_team',
    'away_team': 'kalshi_away_team'
})

combined_rows = []
len_matched = len(matched_names['h2h']['kalshi'])
matched_names_h2h = matched_names['h2h']

for i in range(len_matched):
    odds_name = matched_names_h2h['odds'][i]
    kalshi_name = matched_names_h2h['kalshi'][i]

    # Find the corresponding odds row
    odds_row = odds_subset.loc[odds_subset['team'] == odds_name]
    assert len(odds_row) == 1, f"Expected one row for {odds_name}, got {len(odds_row)}"

    # Find the two matching Kalshi rows
    kalshi_rows = kalshi_subset.loc[
        (kalshi_subset['kalshi_home_team'] == kalshi_name) |
        (kalshi_subset['kalshi_away_team'] == kalshi_name)
    ]
    assert len(kalshi_rows) == 2, f"Expected two rows for {kalshi_name}, got {len(kalshi_rows)}"

    # Extract rows
    k1 = kalshi_rows.iloc[0]
    k2 = kalshi_rows.iloc[1]
    midprice1 = (k1['yes_bid'] + k1['yes_ask']) / 2
    midprice2 = (k2['yes_bid'] + k2['yes_ask']) / 2
   
    # Extract scalar fair probability
    prb = odds_row['avg_fair_prb'].astype(float).item()

    # Choose the row closer to the odds probability
    if ((midprice1 - prb) ** 2) < ((midprice2 - prb) ** 2):
        combined_row = pd.concat([k1, odds_row.iloc[0]])
    else:
        combined_row = pd.concat([k2, odds_row.iloc[0]])

    combined_rows.append(combined_row)

combined_winners_df = pd.DataFrame(combined_rows).sort_values(by='odds_home_team')
combined_winners_df = combined_winners_df.reset_index(drop=True)

In [16]:
combined_winners_df.head()

,ticker,yes_bid,yes_ask,no_bid,no_ask,kalshi_home_team,kalshi_away_team,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb
0,KXNFLGAME-25DEC01NYGNE-NE,0.76,0.77,0.23,0.24,New England,New York G,h2h,2025-12-01 19:15:00 CST,New England Patriots,New England Patriots,New York Giants,0.7639
1,KXNFLGAME-25DEC01NYGNE-NYG,0.24,0.25,0.75,0.76,New England,New York G,h2h,2025-12-01 19:15:00 CST,New York Giants,New England Patriots,New York Giants,0.2361


In [17]:
EDGE = 0.01
KELLY_UPPERBOUND = 1
BANKROLL = 300.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

midprice = (combined_winners_df['yes_bid'] + combined_winners_df['yes_ask']) / 2

edge_winners_df = combined_winners_df.loc[
    (combined_winners_df['avg_fair_prb'] >= midprice + EDGE) |
    (combined_winners_df['avg_fair_prb'] <= midprice - EDGE)
].reset_index(drop=True)

edge_winners_df = edge_winners_df.loc[((edge_winners_df['avg_fair_prb'] > 0.16) & (edge_winners_df['avg_fair_prb'] < 0.49)) |
                                      ((edge_winners_df['avg_fair_prb'] > 0.51) & (edge_winners_df['avg_fair_prb'] < 0.84)) ]

midprice_yes = (edge_winners_df['yes_bid'] + edge_winners_df['yes_ask']) / 2
midprice_no = (edge_winners_df['no_bid'] + edge_winners_df['no_ask']) / 2

q_yes = edge_winners_df['avg_fair_prb']
q_no = 1 - edge_winners_df['avg_fair_prb'] 

edge_winners_df['edge'] = np.where(q_yes > midprice_yes, q_yes - midprice_yes, q_no - midprice_no)

edge_winners_df['buy_direction'] = np.where(q_yes > midprice_yes, "yes", "no")
edge_winners_df['raw_kelly'] = np.where(q_yes > midprice_yes, edge_winners_df['edge'] / (1 - midprice_yes),
                                        edge_winners_df['edge'] / (1 - midprice_no))

total_kelly = edge_winners_df['raw_kelly'].sum() 
if total_kelly >= 1: 
    edge_winners_df['real_kelly'] = pd.DataFrame({
        'original': edge_winners_df['raw_kelly'],
        'normalized': (edge_winners_df['raw_kelly'] / total_kelly)
    }).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.05 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.95:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0 

# Apply to the DataFrame
edge_winners_df['real_kelly'] = edge_winners_df.apply(scale_kelly, axis=1)
edge_winners_df['optimal_bet'] = edge_winners_df['real_kelly'] * BANKROLL

q = edge_winners_df['avg_fair_prb']
p = midprice_yes

num_contracts = np.where(q > p, edge_winners_df['optimal_bet'] // edge_winners_df['yes_bid'], edge_winners_df['optimal_bet'] // edge_winners_df['no_bid'])
edge_winners_df['num_contracts'] = num_contracts
trading_cost = np.where(q > p, np.ceil(100*(0.0175 * num_contracts * edge_winners_df['yes_bid'] * (1 - edge_winners_df['yes_bid']))) / 100,
                        np.ceil(100*(0.0175 * num_contracts * edge_winners_df['no_bid'] * (1 - edge_winners_df['no_bid']))) / 100)
edge_winners_df['trading_cost'] = trading_cost
profit = np.where(q > p, ((1 - edge_winners_df['yes_bid']) *  num_contracts - trading_cost), ((1 - edge_winners_df['no_bid']) *  num_contracts - trading_cost))
edge_winners_df['profit'] = profit
edge_winners_df['ev'] = np.where(q > p, (profit * q_yes - (edge_winners_df['optimal_bet'] + trading_cost) * (1 - q_yes)).round(2), 
                                 (profit * q_no - (edge_winners_df['optimal_bet'] + trading_cost) * (1 - q_no)).round(2))
filtered_winners_df = edge_winners_df.loc[edge_winners_df['ev'] > 0.0].reset_index(drop=True)

s = filtered_winners_df['start_time'].astype(str)
s = s.str.replace(r'\s+[A-Z]{3}$', '', regex=True)
dt = pd.to_datetime(s, errors='coerce')
filtered_winners_df['start_time'] = dt.dt.tz_localize('America/Chicago')

now = datetime.now(pytz.timezone('America/Chicago'))
#filtered_winners_df = filtered_winners_df.loc[filtered_winners_df['start_time'] > now].sort_values('odds_home_team').reset_index(drop=True)

dupe_mask = filtered_winners_df['kalshi_home_team'].duplicated(keep=False)
dupes = filtered_winners_df[dupe_mask]
uniques = filtered_winners_df[~dupe_mask]
best_dupes = dupes.loc[dupes.groupby('kalshi_home_team')['ev'].idxmax()]
filtered_winners_df = pd.concat([uniques, best_dupes], ignore_index=True)

In [18]:
team_cols = ['kalshi_home_team', 'kalshi_away_team',
             'odds_home_team', 'odds_away_team']

teams_df = filtered_winners_df[team_cols].copy()

filtered_winners_df = filtered_winners_df.drop(columns=['kalshi_home_team', 'kalshi_away_team']).reset_index(drop=True)
filtered_winners_df[['edge', 'raw_kelly', 'real_kelly']] = filtered_winners_df[['edge', 'raw_kelly', 'real_kelly']].round(4) * 100

In [19]:
filtered_winners_df

,ticker,yes_bid,yes_ask,no_bid,no_ask,market,start_time,team,odds_home_team,odds_away_team,avg_fair_prb,edge,buy_direction,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev


In [20]:
total_loss = np.sum(filtered_winners_df['optimal_bet'])
total_profit = np.sum(filtered_winners_df['profit'])
total_ev = np.sum(filtered_winners_df['ev'])
print(f"{odds_sport} h2h portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


nfl h2h portfolio summary:

Max Loss: -0.00
Max Profit: 0.00
Portfolio EV: 0.00


shin, median: 
Max Loss: -221.27
Max Profit: 247.18
Portfolio EV: 4.39

shin, mean: Max Loss: -226.99
Max Profit: 217.48
Portfolio EV: 4.34

add, median: Max Loss: -200.15
Max Profit: 448.33
Portfolio EV: 9.09

add, mean: Max Loss: -220.10
Max Profit: 475.53
Portfolio EV: 9.60

shin, median: 

In [21]:
kalshi_cols = ['ticker', 'yes_bid', 'yes_ask', 'team', 'points']
odds_cols = ['market', 'start_time', 'team', 'home_team', 'away_team', 'avg_fair_prb', 'point']

odds_subset = odds_spreads_df[odds_cols].rename(columns={
    'home_team': 'odds_home_team',
    'away_team': 'odds_away_team',
    'team': 'odds_team'
})

kalshi_subset = kalshi_spreads_df[kalshi_cols]

combined_rows = []

for _, kalshi_row in kalshi_subset.iterrows():
    kalshi_home = kalshi_row['team']
    for _, odds_row in odds_subset.iterrows():
        odds_home = odds_row['odds_team']
        if (kalshi_home in odds_home) and (kalshi_row['points'] == odds_row['point']): 
            combined_row = pd.concat([kalshi_row, odds_row])
            combined_rows.append(combined_row)

combined_spreads_df = pd.DataFrame(combined_rows).drop_duplicates(subset='ticker') #only works because oddsapi only pulls odds 
combined_spreads_df = combined_spreads_df.reset_index(drop=True)                    #for only one point line for each bookmaker 



In [22]:
EDGE = 0.01
KELLY_UPPERBOUND = 1
BANKROLL = 100.00
Q1_WEIGHT = 1.00
Q2_WEIGHT = 1.00
Q3_WEIGHT = 1.00
Q4_WEIGHT = 1.00

edge_spreads_df = combined_spreads_df.loc[(combined_spreads_df['avg_fair_prb'] >= combined_spreads_df['yes_ask'] + EDGE) |
                        (combined_spreads_df['avg_fair_prb'] <= combined_spreads_df['yes_bid'] - EDGE)].reset_index(drop=True)

midprice = (edge_spreads_df['yes_bid'] + edge_spreads_df['yes_ask']) / 2

q = edge_spreads_df['avg_fair_prb']
p = midprice

edge_spreads_df['raw_kelly'] = np.where(
    q > p,
    (q - p) / (1 - p),      # long position
    (p - q) / p           # short position
)

total_kelly = edge_spreads_df['raw_kelly'].sum()
edge_spreads_df['raw_kelly'] = pd.DataFrame({
    'original': edge_spreads_df['raw_kelly'],
    'normalized': (edge_spreads_df['raw_kelly'] / total_kelly)
}).min(axis=1)

# Define the real_kelly logic
def scale_kelly(row):
    k = row['raw_kelly']
    p = row['avg_fair_prb']
    
    if k == 0 or pd.isna(k):
        return 0
    if 0.1 <= p < 0.25:
        return min(Q1_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.25 <= p < 0.5:
        return min(Q2_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.5 <= p < 0.75:
        return min(Q3_WEIGHT * k, KELLY_UPPERBOUND)
    elif 0.75 <= p < 0.9:
        return min(Q4_WEIGHT * k, KELLY_UPPERBOUND)
    else:
        return 0  # fallback if out of range

# Apply to the DataFrame
edge_spreads_df['real_kelly'] = edge_spreads_df.apply(scale_kelly, axis=1)
edge_spreads_df['optimal_bet'] = edge_spreads_df['real_kelly'] * BANKROLL

num_contracts = edge_spreads_df['optimal_bet'] // edge_spreads_df['yes_bid']
edge_spreads_df['num_contracts'] = num_contracts
trading_cost = np.ceil(100 * (0.0175 * num_contracts * edge_spreads_df['yes_bid'] * (1 - edge_spreads_df['yes_bid']))) / 100
edge_spreads_df['trading_cost'] = trading_cost
profit = (1 - edge_spreads_df['yes_bid']) *  num_contracts - trading_cost
edge_spreads_df['profit'] = profit
edge_spreads_df['ev'] = (profit * edge_spreads_df['avg_fair_prb'] - (edge_spreads_df['optimal_bet'] + trading_cost) * (1 - edge_spreads_df['avg_fair_prb'])).round(2)

filtered_spreads_df = edge_spreads_df.loc[edge_spreads_df['ev'] > 0].reset_index(drop=True)

s = filtered_spreads_df['start_time'].astype(str)
s = s.str.replace(r'\s+[A-Z]{3}$', '', regex=True)
dt = pd.to_datetime(s, errors='coerce')
filtered_spreads_df['start_time'] = dt.dt.tz_localize('America/Chicago')

now = datetime.now(pytz.timezone('America/Chicago'))
#filtered_spreads_df = filtered_spreads_df.loc[filtered_spreads_df['start_time'] > now].sort_values('odds_home_team').reset_index(drop=True)


In [23]:
# First compute midprice in combined_winners_df
combined_winners_df['midprice'] = (
    combined_winners_df['yes_bid'] + combined_winners_df['yes_ask']
) / 2

# Merge spreads df with winner probabilities
merged = filtered_spreads_df.merge(
    combined_winners_df[['team', 'midprice']],
    left_on='odds_team',      # column in filtered_spreads_df
    right_on='team',          # column in combined_winners_df
    how='left'
)

# Filter out teams with win prob < 50% OR missing (NaN)
filtered_spreads_df = merged.loc[
    merged['midprice'] >= 0.50
].reset_index(drop=True)


In [24]:
filtered_spreads_df

,ticker,yes_bid,yes_ask,team_x,points,market,start_time,odds_team,odds_home_team,odds_away_team,...,point,raw_kelly,real_kelly,optimal_bet,num_contracts,trading_cost,profit,ev,team_y,midprice


In [25]:
total_loss = np.sum(filtered_spreads_df['optimal_bet'])
total_profit = np.sum(filtered_spreads_df['profit'])
total_ev = np.sum(filtered_spreads_df['ev'])
print(f"{odds_sport} spreads portfolio summary:\n")
print(f"Max Loss: -{total_loss:.2f}")
print(f"Max Profit: {total_profit:.2f}")
print(f"Portfolio EV: {total_ev:.2f}")


nfl spreads portfolio summary:

Max Loss: -0.00
Max Profit: 0.00
Portfolio EV: 0.00
